In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/ZUM/Project 1/Основний варіант/')

print(os.listdir('/content/drive/My Drive/ZUM/Project 1/Основний варіант/'))

import pandas as pd

Mounted at /content/drive
['anime_no_index.csv', 'anime', 'war', 'task1_1.ipynb', 'task_1_1_result.csv', 'task_1_1_result.gsheet', 'task_1_2_word2vec.model', 'task_1_2_word2vec.model.wv.vectors.npy', 'task_1_2_word2vec.model.syn1neg.npy', 'task_1_2_sentiment_dictionary.csv', 'task1_2.ipynb', 'task_1_2_labeled_data.csv', 'task2_1.ipynb', 'distilbert-base-uncased_sentiment_analysis', 'task4_1.ipynb']


In [2]:
!pip install -U datasets

!pip install transformers accelerate
!pip install transformers==4.28.0
!pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 99.5 MB/s eta 0:00

In [3]:
import transformers
transformers.__version__

'4.28.0'

In [4]:
df = pd.read_csv('task_1_2_labeled_data.csv')
df.rename(columns={'Tweet':'text', 'prediction':'label'}, inplace=True)
df.sample(10)

,text,label
75491,anime watch ill watch recommendation,1
131260,byzcat islam support femboy,1
136081,shaman king eu tava vendo o remake da netflix ...,0
156836,preciso o anime onde todo mundo bonito,0
99968,white haired man anime type fr,1
15408,people actually marriage pact anime likeif mar...,1
26295,tayrasian lanim de chrno crusade fait un peu n...,1
125011,frasespignavine segu opinando de anim pedazo d...,0
58669,mau ngomong tentang anime,1
72247,extraaba tanto ver anime,0


In [5]:
from datasets import Dataset

dataset_ = Dataset.from_pandas(df[0:5000])
dataset = dataset_.train_test_split(0.1)

In [6]:
model_checkpoint = 'distilbert-base-uncased'
batch_size = 128

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [8]:
def process(x):
  return tokenizer(x['text'])

train_ds = dataset['train'].map(process)
test_ds = dataset['test'].map(process)

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
print(train_ds)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4500
})


In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [11]:
args = TrainingArguments(
    f'{model_checkpoint}_sentiment_analysis',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 5,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy'
)

In [12]:
from datasets import load_metric
import numpy as np

metric = load_metric('glue', 'sst2')

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

<ipython-input-12-35f6af1b57cb>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'sst2')


In [13]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [14]:
trainer.evaluate([train_ds[0]])

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6198843717575073,
 'eval_accuracy': 1.0,
 'eval_runtime': 3.6798,
 'eval_samples_per_second': 0.272,
 'eval_steps_per_second': 0.272}

In [15]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.084699,0.974000
2,No log,0.070535,0.978000
3,No log,0.080983,0.976000
4,No log,0.073734,0.972000
5,No log,0.073192,0.976000


TrainOutput(global_step=180, training_loss=0.10017952389187283, metrics={'train_runtime': 173.2883, 'train_samples_per_second': 129.841, 'train_steps_per_second': 1.039, 'total_flos': 549908393375376.0, 'train_loss': 0.10017952389187283, 'epoch': 5.0})